In [1]:
import requests #Library for sending HTTP requests to the SEC servers
import pandas as pd # Data analysis library for handing tables (DataFrames)
import time # Used to implement "throttling" to avoid being blocked
import json # Used for parsing the JSON data format returned by the SEC
# Panda display configuration (set float format to 4 decimal places)
pd.set_option("display.float_format", lambda x: f"{x:,.4f}")

In [2]:
# SEC EDGAR identity header
SEC_HEADERS = {
    "User-Agent": "FundamentalAgent zcahwya@ucl.ac.uk",
    "Accept-Encoding": "gzip, deflate" # Allows compressed data transfer for faster downloads
}

def sec_get(url, sleep=0.2):
    """
    Fetches JSON data from a specific SEC URL.
    
    Args:
        url (str): The API endpoint (e.g., Company Concept or Company Facts).
        sleep (float): Seconds to pause. Default 0.2s ensures <10 requests/sec limit.
    """
    # Perform the GET request with our custom headers and a 30-second timeout
    r = requests.get(url, headers=SEC_HEADERS, timeout=30)
    
    # Check if the request was successful; raises an exception for 4xx or 5xx errors
    r.raise_for_status()
    
    # Pause execution to prevent overwhelming the server (rate limiting)
    time.sleep(sleep)
    
    # Parse and return the response body as a Python dictionary/list
    return r.json()

def _pick_unit(item):
    """
    Extracts the unit type and the list of data points from a specific financial concept.
    
    Structure of 'item': {"description": "...", "units": {"USD": [{val, fy...}, ...]}}
    """
    # Safely access the "units" dictionary; return empty dict if not found
    units = item.get("units", {})
    
    # If no units exist, the data point is empty/invalid
    if not units:
        return None, None
    
    # If the data is in USD (typical for financial values), return it
    if "USD" in units:
        return "USD", units["USD"]
    
    # If not USD (e.g., "shares" or "pure"), pick the first available unit
    unit = next(iter(units.keys()))
    return unit, units[unit]

In [3]:
# Income Statement & Cash Flow (Annual)
def extract_annual_income_cf(facts, tag):
    """
    Extracts annual (10-K) data for a specific accounting tag from the SEC facts dictionary.
    
    Args:
        facts (dict): The full JSON dictionary returned by the SEC (Company Facts).
        tag (str): The specific GAAP tag to look for (e.g., "NetIncomeLoss").
    """
    item = facts.get("us-gaap", {}).get(tag)
    if not item:
        # Return an empty Pandas Series if the tag doesn't exist for this company
        return pd.Series(dtype=float)

    # Use the previously defined helper to get the unit (e.g., USD) and data array
    _, arr = _pick_unit(item)
    if arr is None:
        return pd.Series(dtype=float)

    # Convert the list of dictionaries into a Pandas DataFrame for processing
    df = pd.DataFrame(arr)

    # Filter for Annual Data ONLY:
    # 'form' == '10-K' ensures we don't include quarterly 10-Q reports.
    # 'fp' == 'FY' ensures we only get the full Fiscal Year results.
    df = df[(df["form"] == "10-K") & (df["fp"] == "FY")].copy()

    # Handle Data Cleaning: Convert 'filed' string to actual datetime objects to allow sorting
    df["filed"] = pd.to_datetime(df["filed"], errors="coerce")
    
    # Sort by fiscal year (fy) and then by filing date. 
    # If a company files an amendment (10-K/A), 'keep="last"' ensures we take 
    # the most recent/corrected version of the data for that year.
    df = df.sort_values(["fy", "filed"]).drop_duplicates("fy", keep="last")

    # Set the Fiscal Year as the index, select the 'val' (value) column, and ensure the index is sorted chronologically.
    s = df.set_index("fy")["val"].astype(float).sort_index()
    s.name = tag # Name the series with the tag for easier table joining later
    return s


In [4]:
# Balance Sheet (Year-End)
def extract_annual_balance_sheet(facts, tag):
    """
    Extracts annual snapshot data (Balance Sheet) for a specific GAAP tag.
    
    Args:
        facts (dict): The full SEC JSON data.
        tag (str): The Balance Sheet tag (e.g., "Assets", "Liabilities").
    """
    item = facts.get("us-gaap", {}).get(tag) # Access the specific GAAP concept
    if not item:
        return pd.Series(dtype=float)

    #Extract units and the raw data array
    _, arr = _pick_unit(item)
    if arr is None:
        return pd.Series(dtype=float)

    df = pd.DataFrame(arr) # Load into a DataFrame
    df = df[df["form"] == "10-K"].copy()  # Filter for Annual filings (10-K)
    # Convert date columns to datetime objects
    df["end"] = pd.to_datetime(df["end"], errors="coerce")
    df["filed"] = pd.to_datetime(df["filed"], errors="coerce")
    df = df.sort_values(["fy", "end", "filed"]).drop_duplicates("fy", keep="last") # Sort value

    # Create the final Series
    s = df.set_index("fy")["val"].astype(float).sort_index()
    s.name = tag
    return s


In [5]:
# Shares Outstanding
def extract_shares_outstanding_year_end(facts):
    """
    Retrieves the annual share count from the SEC 'dei' namespace.
    """
    dei = facts.get("dei", {}) # Access the 'dei' dictionary (Document and Entity Information)
    
    # List of possible tags because companies use different names for shares
    tags = ["EntityCommonStockSharesOutstanding", "CommonStockSharesOutstanding"]

    for tag in tags:
        item = dei.get(tag)
        if not item:
            continue  # If tag not found, try the next one in the list

        units = item.get("units", {}) # Units for shares are not always "shares" (could be "shares", "Shares", etc.)
        for k in units:
            # Look for any unit key that contains the word "share" 
            if "share" in k.lower():
                df = pd.DataFrame(units[k])
                
                # Filter for annual reports:
                # Uses .startswith("10-K") to catch "10-K", "10-K/A" (amendments), etc.
                df = df[df["form"].str.startswith("10-K", na=False)]
                # Sort by fiscal year and filing date, keeping the most recent filing per year.
                df = df.sort_values(["fy", "filed"]).drop_duplicates("fy", keep="last")
                
                # Return a Series indexed by Fiscal Year (fy) with the share count values.
                s = df.set_index("fy")["val"].astype(float).sort_index()
                s.name = "Shares"
                return s
            
    return pd.Series(dtype=float, name="Shares")


In [6]:
# Fetch KO Financials (5Y)
def fetch_KO_financials_5y():
    cik = "0000021344" #Coca-cola company
    data = sec_get(f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json")
    facts = data["facts"]
    
    debt_tags = ["LongTermDebtAndCapitalLeaseObligations", "LongTermDebtNoncurrent",
                 "DebtLongTerm", "NotesPayableLongTerm","ShortTermBorrowings",
                 "DebtCurrent", "LongTermDebtCurrent",]

    # --- Income statement + Cashflow (FY) ---
    Revenue = extract_annual_income_cf(facts, "Revenues")
    if Revenue.empty:
        Revenue = extract_annual_income_cf(facts, "SalesRevenueNet")

    GrossProfit = extract_annual_income_cf(facts, "GrossProfit")
    OperatingIncome = extract_annual_income_cf(facts, "OperatingIncomeLoss")
    NetIncome = extract_annual_income_cf(facts, "NetIncomeLoss")

    OCF = extract_annual_income_cf(facts, "NetCashProvidedByUsedInOperatingActivities")
    Capex = extract_annual_income_cf(facts, "PaymentsToAcquirePropertyPlantAndEquipment")

    # --- Balance sheet (year-end) ---
    TotalAssets = extract_annual_balance_sheet(facts, "Assets")
    TotalEquity = extract_annual_balance_sheet(facts, "StockholdersEquity")

    Cash = extract_annual_balance_sheet(facts, "CashAndCashEquivalentsAtCarryingValue")
    if Cash.empty:
        Cash = extract_annual_balance_sheet(facts, "CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents")
  
    # ===== Total Liabilities (financially consistent definition) =====
    if not TotalAssets.empty and not TotalEquity.empty:
        TotalLiabilities = TotalAssets - TotalEquity
    else:
        TotalLiabilities = pd.Series(dtype=float)
    
    # ===== Total Debt =====
    debt_series = []
    for tag in debt_tags:
        s = extract_annual_balance_sheet(facts, tag)
        if s is not None and not s.empty:
            debt_series.append(s)
        
    if debt_series: # Start with the first debt item and add the others
        TotalDebt = debt_series[0].copy()
        for s in debt_series[1:]:
            TotalDebt = TotalDebt.add(s, fill_value = 0)
    else:
        TotalDebt = pd.Series(dtype = float)
    
    Shares = extract_shares_outstanding_year_end(facts)
  
    # --- Assemble DataFrame by fiscal year index (fy) ---
    df = pd.DataFrame({
        "Revenue": Revenue,
        "GrossProfit": GrossProfit,
        "OperatingIncome": OperatingIncome,
        "NetIncome": NetIncome,
        "TotalAssets": TotalAssets,
        "TotalLiabilities": TotalLiabilities,
        "TotalEquity": TotalEquity,
        "TotalDebt": TotalDebt,
        "OCF": OCF,
        "Capex": Capex,
        "TotalDebt": TotalDebt,
        "Cash": Cash,
        "Shares": Shares,
    }).sort_index()
    
    # Keep last 5 fiscal years
    df = df.tail(6).copy() # Clean up the index
    df.insert(0, "fiscalYear", df.index.astype(int))
    df = df.reset_index(drop = True)
    return df


In [7]:
# Compute financial ratios
financials = fetch_KO_financials_5y()
def compute_ratios(df):
    out = df.copy()
    # Profitability
    out["gross_margin"] = out["GrossProfit"] / out["Revenue"]
    out["operating_margin"] = out["OperatingIncome"] / out["Revenue"]
    out["net_margin"] = out["NetIncome"] / out["Revenue"]
    out["ROA"] = out["NetIncome"] / out["TotalAssets"]
    out["ROE"] = out["NetIncome"] / out["TotalEquity"]
    # Liquidity & Leverage
    out["debt_to_assets"] = out["TotalLiabilities"] / out["TotalAssets"]
    out["debt_to_equity"] = out["TotalDebt"] / out["TotalEquity"]
    out["Leverage"] = out["TotalAssets"] / out["TotalEquity"]
    # Efficiency
    out["AssetTurnover"] = out["Revenue"] / out["TotalAssets"]
    # Cash flow
    out["fcf"] = out["OCF"] - out["Capex"]
    out["fcf_margin"] = out["fcf"] / out["Revenue"]
    return out.tail(5)

ratios = compute_ratios(financials)
ratio_cols = [ "fiscalYear", "gross_margin", "operating_margin", "net_margin", "ROA", "ROE",
            "debt_to_assets", "debt_to_equity", "Leverage", "AssetTurnover",  "fcf", "fcf_margin"]
ratios[ratio_cols]

,fiscalYear,gross_margin,operating_margin,net_margin,ROA,ROE,debt_to_assets,debt_to_equity,Leverage,AssetTurnover,fcf,fcf_margin
1,2020,0.5931,0.2725,0.2347,0.0887,0.4014,0.7789,2.2174,4.5233,0.3782,"8,667,000,000.0000",0.2625
2,2021,0.6027,0.2667,0.2528,0.1036,0.4248,0.7562,1.8593,4.1025,0.4097,"11,258,000,000.0000",0.2912
3,2022,0.5814,0.2537,0.2219,0.1029,0.3959,0.7401,1.6241,3.8483,0.4636,"9,534,000,000.0000",0.2217
4,2023,0.5952,0.2472,0.2342,0.1097,0.4130,0.7345,1.4459,3.7664,0.4683,"9,747,000,000.0000",0.2130
5,2024,0.6106,0.2123,0.2259,0.1057,0.4277,0.7528,1.7048,4.0453,0.4680,"4,741,000,000.0000",0.1007


In [8]:
# DCF Valuation
DCF_PARAMS = {
    "projection_years": 5, # forecast 5 years
    "fcf_growth_start": 0.06, # Starting growth rate of free cash flow
    "fcf_growth_end": 0.03, # Growth rate at the end of the projection
    "discount_rate": 0.08, # Discount rate
    "terminal_growth": 0.025 # Perpetual growth rate after year 5
}
def dcf_valuation_from_financials(financials, params):
    # Calculate basic FCF
    last = financials.iloc[-1]
    base_fcf = last["OCF"] - last["Capex"] 
    r, g, n = params["discount_rate"], params["terminal_growth"], params["projection_years"]

    fcfs, pvs = [], []
    fcf = base_fcf
    # Future FCF and Present Value
    for t in range(1, n + 1):
        # Growth rate decreases linearly each year
        fcf *= 1 + (params["fcf_growth_start"] +
                    (params["fcf_growth_end"] - params["fcf_growth_start"]) * (t - 1) / (n - 1))
        pvs.append(fcf / (1 + r) ** t)
        fcfs.append(fcf)
    # Terminal Value & Enterprise Value
    terminal = fcfs[-1] * (1 + g) / (r - g)
    equity = sum(pvs) + terminal - (last["TotalLiabilities"] - last["Cash"])
    per_share = equity / last["Shares"]

    return {
        "intrinsic_value_per_share": per_share,
        "equity_value": equity
    }

dcf_result = dcf_valuation_from_financials(financials, DCF_PARAMS)

dcf_df = pd.DataFrame.from_dict(
    dcf_result,
    orient="index",
    columns=["value"]
)


dcf_df.loc["equity_value", "value"] /= 1e9 # Convert to billions for better display
dcf_df.rename(
    index={"equity_value": "Equity Value (USD bn)",
           "intrinsic_value_per_share": "Intrinsic Value per Share (USD)"},
    inplace=True
)

dcf_df
# Show the results
summary = {
    "Intrinsic Value per Share (USD)": dcf_result["intrinsic_value_per_share"],
    "Equity Value (USD bn)": dcf_result["equity_value"] / 1e9,
    "Discount Rate": DCF_PARAMS["discount_rate"],
    "Terminal Growth Rate": DCF_PARAMS["terminal_growth"],
    "Projection Years": (DCF_PARAMS["projection_years"])
}

pd.DataFrame.from_dict(summary, orient="index", columns=["Value"])

,Value
Intrinsic Value per Share (USD),15.5833
Equity Value (USD bn),67.0238
Discount Rate,0.0800
Terminal Growth Rate,0.0250
Projection Years,5.0000


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_API_KEY"]

# Build LLM Input
def build_llm_input(financials, ratios, dcf):
    """
    Combines all processed data into a single dictionary for the LLM.
    """
    return {
        "company": "The Coca-Cola Company",
        "ticker": "KO",
        # .to_dict(orient="records") converts the DataFrame into a list of 
        # dictionaries (one per year), which is highly readable for AI.
        "financials_5y": financials.to_dict(orient="records"),
        "ratios_5y": ratios.to_dict(orient="records"),
        "valuation": dcf,
        "data_sources": [
            "SEC EDGAR companyfacts API",
            "US-GAAP 10-K filings"
        ]
    }


# LLM Policy
INVESTMENT_MEMO_POLICY = """
You are a professional buy-side equity analyst.
Generate a 1–2 page investment memo strictly based on the provided data, and give the investment advise(Buy, Sell or Hold).
Do not perform calculations or use external information.
Follow standard investment memo structure and cite data sources.
"""
from openai import OpenAI
# Generate Investment Memo (LLM Agent)
client = OpenAI()

def generate_investment_memo(llm_input):
    """
    Sends the packaged data to GPT-4o-mini and returns the professional memo.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini", # A fast, cost-effective model for data synthesis
        messages=[
            {"role": "system", "content": INVESTMENT_MEMO_POLICY},
            {"role": "user", "content": json.dumps(llm_input, indent=2)}
        ],
        # Temperature 0.2: Low temperature makes the AI more focused and 
        # mathematically consistent, rather than "creative."
        temperature=0.2 
    )
    # Extract the text content of the AI's response
    return response.choices[0].message.content


In [10]:
# Run The Full Agent
financials = fetch_KO_financials_5y()
ratios = compute_ratios(financials) # Quantitative analysis
dcf = dcf_valuation_from_financials(financials, DCF_PARAMS) # DCF Valuation modeling
# Data packing
llm_input = build_llm_input(financials, ratios, dcf)
# Send the package to GPT-4o-mini to write the final professional memo
memo = generate_investment_memo(llm_input)

print(memo[:1000])  # preview

financials.to_csv("KO_statements_5y.csv", index=False) # Save raw financial statments
ratios.to_csv("KO_ratios_5y.csv", index=False) # Save the calculated ratios
# Save the AI-generated report as a Markdown file
with open("KO_investment_memo.md", "w") as f:
    f.write(memo)

print("All outputs saved.")

**Investment Memo: The Coca-Cola Company (KO)**

**Date:** October 2023  
**Prepared by:** [Your Name]  
**Position:** Buy-Side Equity Analyst  

---

**Company Overview**  
The Coca-Cola Company (KO) is a leading global beverage company, known for its iconic soft drinks and a diverse portfolio of non-alcoholic beverages. The company operates in more than 200 countries and has a strong brand presence, making it a staple in the consumer goods sector.

---

**Financial Performance**  
Over the past five years, Coca-Cola has demonstrated a solid financial trajectory:

- **Revenue Growth:** Revenue has increased from $37.27 billion in 2019 to an estimated $47.06 billion in 2024, reflecting a compound annual growth rate (CAGR) of approximately 6.4%. This growth is indicative of the company's ability to adapt to changing consumer preferences and expand its product offerings.
  
- **Profitability Metrics:** The gross margin has remained relatively stable, increasing from 60% in 2020 to approx